In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

# -------------------------
# データ生成
# -------------------------
np.random.seed(42)
T = 2000  # データ長
t = np.arange(T)
# サイン波 + 雑音
data = np.sin(2 * np.pi * t / 50) + 0.5 * np.random.randn(T)

# 訓練・テスト分割
train_ratio = 0.8
train_size = int(T * train_ratio)

train_data = data[:train_size]
test_data = data[train_size:]

# horizon=1を想定: y(t+1)を予測
# -------------------------
# naive GBRT: 単純な特徴量（ここでは時点tのみ）
# -------------------------
# naiveモデル: 特徴量はここでは簡略的に t のインデックスそのもの（あるいは data(t)）を使う
# 実際の予測では単純すぎるが、比較目的の例示として用いる。
X_train_naive = train_data[:-1].reshape(-1, 1)  # 特徴量: data(t)
y_train_naive = train_data[1:]                 # ターゲット: data(t+1)

X_test_naive = test_data[:-1].reshape(-1, 1)
y_test_naive = test_data[1:]

# naive GBRTモデルの学習
model_naive = XGBRegressor(n_estimators=100, random_state=42)
model_naive.fit(X_train_naive, y_train_naive)

y_pred_naive = model_naive.predict(X_test_naive)

rmse_naive = np.sqrt(mean_squared_error(y_test_naive, y_pred_naive))
mae_naive = mean_absolute_error(y_test_naive, y_pred_naive)

# -------------------------
# wbgbrt: ウィンドウ化した特徴量を使用
# -------------------------
window_size = 24  # 例として24ステップの履歴を使う
# 過去window_size個の値を1サンプルとして、次の値を予測
def make_windowed_data(series, window):
    X = []
    y = []
    for i in range(len(series) - window):
        X.append(series[i:i+window])
        y.append(series[i+window])
    return np.array(X), np.array(y)

X_train_wb, y_train_wb = make_windowed_data(train_data, window_size)
X_test_wb, y_test_wb = make_windowed_data(test_data, window_size)

# wbgbrtモデルの学習
model_wb = XGBRegressor(n_estimators=100, random_state=42)
model_wb.fit(X_train_wb, y_train_wb)

y_pred_wb = model_wb.predict(X_test_wb)

rmse_wb = np.sqrt(mean_squared_error(y_test_wb, y_pred_wb))
mae_wb = mean_absolute_error(y_test_wb, y_pred_wb)

# -------------------------
# 結果表示
# -------------------------
print("=== Naive GBRT ===")
print(f"RMSE: {rmse_naive:.4f}")
print(f"MAE:  {mae_naive:.4f}")

print("\n=== Window-based GBRT (wbgbrt) ===")
print(f"Window Size: {window_size}")
print(f"RMSE: {rmse_wb:.4f}")
print(f"MAE:  {mae_wb:.4f}")


=== Naive GBRT ===
RMSE: 0.6845
MAE:  0.5350

=== Window-based GBRT (wbgbrt) ===
Window Size: 24
RMSE: 0.5670
MAE:  0.4510


In [2]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from xgboost import XGBRegressor

# -------------------------
# データ生成
# -------------------------
np.random.seed(42)
T = 2000  # データ長
t = np.arange(T)
data = np.sin(2 * np.pi * t / 50) + 0.5 * np.random.randn(T)  # サイン波＋ノイズ

# -------------------------
# ウィンドウデータの作成関数
# -------------------------
def make_windowed_data(series, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(series[i + window_size])
    return np.array(X), np.array(y)

# -------------------------
# 時系列交差検証の準備
# -------------------------
window_size = 24  # 過去24ステップ分を特徴量として使用
X, y = make_windowed_data(data, window_size)

# TSCV分割: スライディングウィンドウ法を実装
n_splits = 5  # 分割数
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=100)  # テストセットのサイズを指定

# -------------------------
# モデル評価（交差検証）
# -------------------------
rmse_list = []
mae_list = []

for train_index, test_index in tscv.split(X):
    # 訓練セットとテストセットを分割
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # モデルの初期化と学習
    model = XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # テストセットでの予測と評価
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # スコアの記録
    rmse_list.append(rmse)
    mae_list.append(mae)

# -------------------------
# 評価結果の出力
# -------------------------
print("=== Time Series Cross-Validation Results ===")
print(f"Average RMSE: {np.mean(rmse_list):.4f} ± {np.std(rmse_list):.4f}")
print(f"Average MAE:  {np.mean(mae_list):.4f} ± {np.std(mae_list):.4f}")


=== Time Series Cross-Validation Results ===
Average RMSE: 0.5791 ± 0.0396
Average MAE:  0.4603 ± 0.0241


In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

# -------------------------
# データ生成
# -------------------------
np.random.seed(42)
T = 2000  # データ長
t = np.arange(T)
data = np.sin(2 * np.pi * t / 50) + 0.5 * np.random.randn(T)  # サイン波 + ノイズ

# -------------------------
# スライディングウィンドウ方式の交差検証
# -------------------------
train_size = 1000  # 訓練データのサイズ
test_size = 100    # テストデータのサイズ
step_size = 100    # スライド幅
window_size = 24   # 特徴量として使用する過去データの長さ

# 特徴量とターゲットのウィンドウ化関数
def make_windowed_data(series, window):
    X, y = [], []
    for i in range(len(series) - window):
        X.append(series[i:i + window])
        y.append(series[i + window])
    return np.array(X), np.array(y)

# 全データのウィンドウ化
X, y = make_windowed_data(data, window_size)

# モデル評価結果を格納
rmse_list = []
mae_list = []

# スライディングウィンドウで分割とモデル評価
start = 0
while start + train_size + test_size <= len(X):
    # 訓練データとテストデータの分割
    train_end = start + train_size
    test_end = train_end + test_size
    X_train, y_train = X[start:train_end], y[start:train_end]
    X_test, y_test = X[train_end:test_end], y[train_end:test_end]
    
    # モデルの学習
    model = XGBRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # テストデータでの予測と評価
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # スコアの記録
    rmse_list.append(rmse)
    mae_list.append(mae)
    
    # スライド
    start += step_size

# -------------------------
# 評価結果の出力
# -------------------------
print("=== Sliding Window Cross-Validation Results ===")
print(f"Average RMSE: {np.mean(rmse_list):.4f} ± {np.std(rmse_list):.4f}")
print(f"Average MAE:  {np.mean(mae_list):.4f} ± {np.std(mae_list):.4f}")


=== Sliding Window Cross-Validation Results ===
Average RMSE: 0.5778 ± 0.0411
Average MAE:  0.4685 ± 0.0404
